# Import Libraries

In [ ]:
!pip install langchain openai
!pip install langchain-community langchain-core

In [ ]:
import os
import langchain
import openai
from pprint import pprint

# Set-up Open-AI Key

In [ ]:
openai_key = "#######" #replace the key with your OpenAI Key
os.environ['OPENAI_API_KEY'] = openai_key
openai.api_key = os.environ['OPENAI_API_KEY']

# Model

Chat models are a variation on language models. While chat models use language models under the hood, the interface they expose is a bit different. Rather than expose a "text in, text out" API, they expose an interface where "chat messages" are the inputs and outputs.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

## Chat Models

Chat Models are the models that are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.

In [ ]:
# To control the randomness and creativity of the generated, use temperature = 0.0 to 0.9
chat = ChatOpenAI(temperature=0.2,max_tokens = 500)

In [ ]:
chat.model_name

'gpt-3.5-turbo'

In [ ]:
print(chat.max_tokens)

500


In [ ]:
llm = OpenAI(model_name='text-davinci-003',temperature=0)

In [ ]:
llm.model_name

'text-davinci-003'

In [ ]:
llm.max_tokens

256

# Prompt Template

LangChain offers tools for crafting and utilizing prompt templates, which are pre-defined guidelines for generating prompts for language models, including instructions, few-shot examples, and context-specific questions suitable for various tasks. These templates aim to be model-agnostic, allowing for easy reuse across different language models. Generally, language models anticipate prompts to be either a string or a series of chat messages

In [ ]:
from langchain.prompts import ChatPromptTemplate

Let's create a prompt template, notice the variables inside {} curly brackets are treated as input variables and can be overriden based on the user input

In [ ]:
template = """Generate a report highlighting the key findings from the {analysis_type} analysis. Here's a snippet of the analysis: \
```{analysis_snippet}```.
"""

In [ ]:
prompt_temp = ChatPromptTemplate.from_template(template)

In [ ]:
pprint(prompt_temp)

ChatPromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], template="Generate a report highlighting the key findings from the {analysis_type} analysis. Here's a snippet of the analysis: ```{analysis_snippet}```.\n"))])


In [ ]:
prompt_temp.input_variables

['analysis_snippet', 'analysis_type']

In [ ]:
prompt_temp.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], template="Generate a report highlighting the key findings from the {analysis_type} analysis. Here's a snippet of the analysis: ```{analysis_snippet}```.\n"))]

In [ ]:
sample_analysis_type = "user_engagement"

sample_analysis_snippet = """
Over the past quarter, the user engagement metrics have shown promising growth.
The number of active users has increased by 15%, with a notable uptick in daily logins.
This increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.
Additionally, user interactions with key features, such as commenting and sharing, have risen by 20%.
This signifies a higher level of user participation and social engagement within our platform.
We've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.
The engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.
This suggests that our efforts to improve user experience and introduce new features are resonating well with our audience.

Overall, the data suggests that our focus on enhancing user engagement through product improvements and feature launches is yielding positive results. As we continue to refine our strategies and innovate, we can expect further growth in user interactions and overall engagement.
"""

Using format_messages function we can override the input variables used in the Prompt template

In [ ]:
response_message = prompt_temp.format_messages(
    analysis_type = sample_analysis_type,
    analysis_snippet = sample_analysis_snippet
)

In [ ]:
response_message

[HumanMessage(content="Generate a report highlighting the key findings from the user_engagement analysis. Here's a snippet of the analysis: ```\nOver the past quarter, the user engagement metrics have shown promising growth.\nThe number of active users has increased by 15%, with a notable uptick in daily logins.\nThis increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.\nAdditionally, user interactions with key features, such as commenting and sharing, have risen by 20%.\nThis signifies a higher level of user participation and social engagement within our platform.\nWe've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.\nThe engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.\nThis suggests that our efforts to improve user experience and introduce new fe

In [ ]:
response_message[0].content

"Generate a report highlighting the key findings from the user_engagement analysis. Here's a snippet of the analysis: ```\nOver the past quarter, the user engagement metrics have shown promising growth.\nThe number of active users has increased by 15%, with a notable uptick in daily logins.\nThis increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.\nAdditionally, user interactions with key features, such as commenting and sharing, have risen by 20%.\nThis signifies a higher level of user participation and social engagement within our platform.\nWe've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.\nThe engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.\nThis suggests that our efforts to improve user experience and introduce new features are resonating 

In [ ]:
report = chat(response_message) #chatopenai model-GPT3.5 turbo

In [ ]:
report_llm = llm(response_message[0].content) #OpenAI -llms - Text davinci-003

In [ ]:
print(report.content)

In [ ]:
print(report_llm)

#Output in specific format and not string

When developing an application, there may be a requirement for the response to adhere to a specific format, such as JSON or XML

In [ ]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}
"""

In [ ]:
movie_bot = ChatPromptTemplate.from_template(movie_review_template)
movie_bot

ChatPromptTemplate(input_variables=['review_text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review_text'], template="For the following movie review, extract the following information:\n\ngenre: Mentioned genres in the review. Extract and list them as a comma separated Python list.\n\nlead_chemistry: Describe the chemistry between the lead characters, if mentioned.\n\nplot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.\n\nruntime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.\n\nFormat the output as JSON with the following keys:\ngenre\nlead_chemistry\nplot_twists\nruntime_feel\n\nreview_text: {review_text}\n"))])

In [ ]:
movie_bot.input_variables

['review_text']

In [ ]:
movie_review = """\
This movie was an incredible cinematic experience. It offers a unique blend of genres:\
romance, drama, and thrilling action. The storyline is gripping, and the characters'\
emotional journey kept me engaged throughout. The visuals are stunning, especially\
during the breathtaking action sequences.

The acting was exceptional, with the lead actors delivering performances that truly\
captured the essence of their roles. The chemistry between the main characters added\
depth to the romantic elements of the plot. The supporting cast also deserves praise,\
as they brought their characters to life convincingly.

I was pleasantly surprised by the unexpected plot twists that kept me on the edge of\
my seat. The pacing was well-managed, allowing for a perfect balance between intense\
action and quieter, reflective moments.

While the movie was a bit longer than usual, I found myself immersed in the story\
and didn't mind the extended runtime. It's a film that leaves a lasting impression,\
prompting viewers to reflect on its themes long after the credits roll.
"""

In [ ]:
delegate_review = movie_bot.format_messages(review_text = movie_review)

In [ ]:
delegate_review

[HumanMessage(content="For the following movie review, extract the following information:\n\ngenre: Mentioned genres in the review. Extract and list them as a comma separated Python list.\n\nlead_chemistry: Describe the chemistry between the lead characters, if mentioned.\n\nplot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.\n\nruntime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.\n\nFormat the output as JSON with the following keys:\ngenre\nlead_chemistry\nplot_twists\nruntime_feel\n\nreview_text: This movie was an incredible cinematic experience. It offers a unique blend of genres:romance, drama, and thrilling action. The storyline is gripping, and the characters'emotional journey kept me engaged throughout. The visuals are stunning, especiallyduring the breathtaking action sequences.\n\nThe acting was exceptional, with the lead actors delivering performances that tr

In [ ]:
get_movie_details = chat(delegate_review)

As you can see the output is in the json format but the data type is still string

In [ ]:
print(get_movie_details.content)

In [ ]:
get_movie_details.get("genre")  # This will resutlt into error which can be solved using Output Parser

# Output Parser
Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

*   "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
*   "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

StructuredOutputParser output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, we initially experimented with data structures having text fields only.

To create such multiple data structures we use ResponseSchema

In [ ]:
genre_schema = ResponseSchema(name = "genre",
description = "Mentioned genres in the review. \
Extract and list them as comma separated Python list.")

lead_chemistry_schema = ResponseSchema(name="lead_chemistry",
description="Describe the chemistry between \
the lead characters, if mentioned.")

plot_twists_schema = ResponseSchema(name="plot_twists",
description="Were any unexpected plot twists mentioned? \
Answer True if yes, False if not or unknown.")

runtime_feel_schema = ResponseSchema(name="runtime_feel",
description="Did the reviewer mention their feelings about \
the movie's runtime? If yes, extract their sentiment about it.")

response_schemas = [genre_schema,
lead_chemistry_schema,
plot_twists_schema,
runtime_feel_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"genre": string  // Mentioned genres in the review. Extract and list them as comma separated Python list.
	"lead_chemistry": string  // Describe the chemistry between the lead characters, if mentioned.
	"plot_twists": string  // Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.
	"runtime_feel": string  // Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.
}
```


Note: Use this format_instructions in your prompt template to get the response in JSON

In [ ]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}

{format_instructions}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=movie_review_template)

In [ ]:
user_review = prompt.format_messages(
review_text=movie_review,
format_instructions=format_instructions
)

In [ ]:
response = chat(user_review)

In [ ]:
print(response.content)

In [ ]:
final_ans = output_parser.parse(response.content)

In [ ]:
final_ans

In [ ]:
final_ans.get("genre")

# Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [ ]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}
"""

In [ ]:
user_prompt = ChatPromptTemplate.from_template(movie_review_template)

In [ ]:
chat = ChatOpenAI(temperature=0.0) #ChatOPENAI

# LLM Chain

An LLMChain is a simple chain that adds some functionality around language models. It is used widely throughout LangChain, including in other chains and agents.

In [ ]:
chain = LLMChain(llm=chat,prompt=user_prompt,verbose=True) # by default - Verbose=False
review_res = chain.run(movie_review)

In [ ]:
print(review_res)

## Sequential Chain

In [ ]:
chain_one = LLMChain(llm=chat,prompt=user_prompt,output_key = "json_answer") #outputkey can be any name

In [ ]:
summary_review = "Please summarize the following review in 1 liner: {review_text}"
second_prompt_template = ChatPromptTemplate.from_template(summary_review)

In [ ]:
chain_two = LLMChain(llm =chat,prompt=second_prompt_template,output_key = "summary")

In [ ]:
seq_chain = SequentialChain(
chains=[chain_one, chain_two],
input_variables=["review_text"],
output_variables=["json_answer", "summary"],
verbose=True
)

In [ ]:
seq_response = seq_chain(movie_review)

In [ ]:
seq_response

# Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [ ]:
temp1 = ChatPromptTemplate.from_template("Hi, my name is {name}, I work at AI Planet")
temp1 = temp1.format_messages(name="Tarun Jain")
res = chat(temp1)

In [ ]:
res

In [ ]:
temp2 = ChatPromptTemplate.from_template("As you know I work at {startup}. What is my name?")
temp2 = temp2.format_messages(startup="AI Planet")
res2 = chat(temp2)

In [ ]:
res

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
conversation = ConversationChain(
llm=chat,
memory = memory,
verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Tarun Jain, I work at AI Planet")

In [ ]:
conversation.predict(input="At AI Planet I work as a DevRel & Community Manager")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

# Agents

Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

In [2]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.1 MB/s eta 0:00:00


In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import initialize_agent

In [ ]:
search = DuckDuckGoSearchAPIWrapper()
tools = [
Tool(
name = "Search Engine",
func=search.run,
description="To explore the world of internet"
),
]

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(openai_api_key=openai_key, temperature=0.0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [ ]:
agent_chain.run(input="Where is Bangalore located?")